In [5]:
import pandas as pd

path = '/home/kmcampbell/Desktop/yelp/data/enhanced_clean.csv'
out =  '/home/kmcampbell/Desktop/yelp/data/summarized.csv'

df = pd.read_csv(path)

# Format FIPS correctly
df['FIPS'] = df['FIPS'].apply(lambda x: str(x).zfill(5))

df.head()

,alias,name,categories,generaldentistry,physicians,c_and_mh,urgent_care,pharmacy,hospitals,price,rating,review_count,address,zipcode,state,lat,lon,FIPS,county
0,southern-comfort-dental-prattville,Southern Comfort Dental,generaldentistry,1,0,0,0,0,0,NaN,5.0,5,"660 McQueen Smith Rd N Ste H Prattville, AL 3...",36066,AL,32.480140,-86.419700,01001,Autauga
1,pickett-family-chiropractic-prattville-2,Pickett Family Chiropractic,chiropractors,0,0,0,0,0,0,NaN,5.0,3,"802 S Memorial Dr Prattville, AL 36067",36067,AL,32.458298,-86.450157,01001,Autauga
2,american-family-care-prattville-prattville,American Family Care Prattville,"familydr, urgent_care, medcenters",0,0,0,1,0,0,NaN,3.5,6,"1965 Cobbs Ford Rd Prattville, AL 36066",36066,AL,32.460277,-86.413146,01001,Autauga
3,first-choice-healthcare-prattville,First Choice Healthcare,"chiropractors, physicaltherapy",0,0,0,0,0,0,NaN,5.0,2,"550 McQueen Smith Rd N Prattville, AL 36066",36066,AL,32.460152,-86.418129,01001,Autauga
4,eyecare-plus-prattville,EyeCare Plus,"optometrists, opticians",0,0,0,0,0,0,NaN,3.5,3,"605 Jasmine Trl Prattville, AL 36066",36066,AL,32.479856,-86.428767,01001,Autauga


In [6]:
grouped = df.groupby(['FIPS', 'county', 'state']).agg({'lon': ['count'],
                                  'rating': ['mean'],
                                  'price': ['count'],
                                  'review_count': ['sum'],
                                  'generaldentistry': ['sum'],
                                  'c_and_mh': ['sum'],
                                  'urgent_care':['sum'],
                                  'pharmacy': ['sum'],
                                  'hospitals': ['sum']})

grouped.columns = ["".join(x) for x in grouped.columns.ravel()]
grouped.head()

,,,ratingmean,review_countsum,generaldentistrysum,hospitalssum,pricecount,c_and_mhsum,loncount,urgent_caresum,pharmacysum
FIPS,county,state,,,,,,,,,
01001,Autauga,AL,3.500000,41,3,0,2,0,19,1,0
01003,Baldwin,AL,3.818182,15,1,0,0,0,11,0,0
01005,Barbour,AL,3.375000,6,1,0,1,0,4,0,0
01007,Bibb,AL,5.000000,1,0,0,0,0,1,0,0
01009,Blount,AL,2.571429,9,0,1,0,0,7,2,0


In [7]:
grouped = grouped.rename(index=str, columns={'ratingmean': 'avg_rating', 'review_countsum': 'total_reviews', 'generaldentistrysum': 'total_generaldentistry',
                                   'hospitalssum': 'total_hospital', 'pricecount': 'total_hasprice', 'c_and_mhsum': 'total_mentalhealth', 'loncount': 'total',
                                   'urgent_caresum': 'total_urgentcare', 'pharmacysum': 'total_pharmacy'})

grouped = grouped[['total', 'avg_rating', 'total_hasprice', 'total_reviews', 'total_generaldentistry', 'total_mentalhealth', 'total_urgentcare', 'total_pharmacy', 'total_hospital']].reset_index()

grouped.to_csv(out)

grouped.head()

,FIPS,county,state,total,avg_rating,total_hasprice,total_reviews,total_generaldentistry,total_mentalhealth,total_urgentcare,total_pharmacy,total_hospital
0,01001,Autauga,AL,19,3.500000,2,41,3,0,1,0,0
1,01003,Baldwin,AL,11,3.818182,0,15,1,0,0,0,0
2,01005,Barbour,AL,4,3.375000,1,6,1,0,0,0,0
3,01007,Bibb,AL,1,5.000000,0,1,0,0,0,0,0
4,01009,Blount,AL,7,2.571429,0,9,0,0,2,0,1


In [12]:
# libraries
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
 
# initialise
map = Basemap(llcrnrlon=-130, llcrnrlat=25, urcrnrlon=-65.,urcrnrlat=52.,resolution='i', lat_0 = 40., lon_0 = -80)
 
# show counties
map.drawmapboundary(fill_color='#A6CAE0')
map.fillcontinents(color='#e6b800',lake_color='#A6CAE0')
map.drawcounties()
map.fillcounties()
plt.show()


TypeError: drawcounties() got an unexpected keyword argument 'fill_color'

In [6]:
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
import geopandas
import shapely
import pyshp

colorscale = ["#f7fbff","#ebf3fb","#deebf7","#d2e3f3","#c6dbef","#b3d2e9","#9ecae1",
              "#85bcdb","#6baed6","#57a0ce","#4292c6","#3082be","#2171b5","#1361a9",
              "#08519c","#0b4083","#08306b"]

endpts = list(np.linspace(1, 12, len(colorscale) - 1))
fips = grouped['FIPS'].tolist()
values = grouped['total'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9, title='USA by Unemployment %',
    legend_title='% unemployed'
)
py.iplot(fig, filename='choropleth_full_usa')

ImportError: libgdal.so.20: cannot open shared object file: No such file or directory